In [1]:
from flask import Flask, render_template, request, jsonify
from dotenv import load_dotenv
import os
import requests
import re
from IPython.display import Markdown, display

In [2]:
load_dotenv()

# Retrieve API keys and configuration from environment variables
HEYGEN_API_KEY = os.getenv("HEYGEN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
VECTOR_STORE_ID = os.getenv("VECTOR_STORE_ID")

In [29]:
def vector_store_search_check(user_input):
    """
    Determine if the user's input should trigger a vector store search.

    This function sends the user's input and a set of instructions
    to the OpenAI API (using a specialized 'gpt-4.1-mini-2025-04-14' model)
    to receive a simple 'ja' or 'nee' response. 'Ja' indicates that:
        1. Specific information is requested.
        2. It is a substantive question about a topic.
        3. Clarification or explanation is requested.
        4. The content relates to any of the listed subjects.

    Returns True if 'ja' was returned by the model, otherwise False.

    Args:
        user_input (str): The text input from the user.

    Returns:
        bool: True if a vector store search should be performed, False otherwise.
    """
    search_check_instructions = (
        f"""
        Je bent een AI die uitsluitend reageert met "ja" of "nee", op basis van de volgende strikte regel:

        Antwoord "ja" als de vraag of opmerking inhoudelijk of taakgericht is (bijvoorbeeld over feiten, opdrachten, uitleg, hulpvragen).
        
        Antwoord "nee" als de vraag of opmerking small talk of sociaal van aard is (bijvoorbeeld begroetingen, beleefdheidsvragen, persoonlijke opmerkingen).
        
        Gebruik uitsluitend het woord "ja" of "nee", zonder verdere toelichting of variatie. Geen uitzonderingen.
        """
    )
    vector_store_search_model = "gpt-4.1-mini-2025-04-14"
    payload = {
        "model": vector_store_search_model,
        "input": user_input,
        "instructions": search_check_instructions
    }
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    openai_url = "https://api.openai.com/v1/responses"

    try:
        # Send the request to OpenAI to get 'ja' or 'nee'
        response = requests.post(openai_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()
        output_text = data['output'][-1]['content'][0]['text']
        print(output_text)
        # Check if 'ja' is present in the response (case-insensitive)
        if re.search(r'\bja\.?\b', output_text, re.IGNORECASE):
            return True
        else:
            return False

    except requests.RequestException:
        # In case of an error with the request, default to False
        return False
print(vector_store_search_check('bedankt voor het lesplan, kan je hem nog wat aanpassen?'))
print('okk')

ja
True
okk


In [53]:
def vector_store_search(query):
    endpoint = f"https://api.openai.com/v1/vector_stores/{VECTOR_STORE_ID}/search"
    payload = {
        "query": query,
        "max_num_results": 10,
        "rewrite_query": True,
        "ranking_options": {
            "score_threshold": 0.7
        }

    }
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    response.raise_for_status()
    context = "Dit zijn de bronnen waarop je het antwoord moet baseren: \n "

    if len(response.json()['data']) == 0:
        return '\nzeg dat je het niet weet omdat je hier geen informatie over hebt en dat ik iets anders kan vragen', []
    
    file_names = []
    for i, result in enumerate(response.json()['data']):
        content = f"Start Bron '{result['filename']}': \n\n{result['content'][0]['text']} \n\n Einde Bron '{result['filename']}' \n\n"
        context += content
        if result['filename'] not in file_names:
            file_names.append(result['filename'])
    return context, file_names




def custom_rag(user_input):
    imce_instructions = (
        """
        Je bent een digitale assistent in de onderwijssector. Je naam is Ella, wat staat voor Education & Learning Assistant. Je bent ontwikkeld door ManageMind Group. Je bronnen zijn zorgvuldig geselecteerd en afkomstig van RIF research en ManageMind group.

        Je hebt 2 kerntaken: het beantwoorden van vakinhoudelijke vragen en het ontwerpen en ontwikkelen van modules met bijpassende lesmiddelen en leermiddelen. 
        
        1. Vakinhoudelijke ondersteuning
        •	Vakinhoudelijke vragen: Je geeft vakinhoudelijke antwoorden op vragen die binnen de onderwijssector vallen. Je combineert verschillende bronnen uit je bibliotheek om een volledig en correct antwoord te formuleren.
        •	Bronverwijzing: Wanneer je een antwoord geeft, verwijs je naar de gebruikte bronnen zodat de gebruiker deze kan raadplegen.
        •	Beperking tot vakinhoud: gebruik voor je antwoord alleen gegeven bronnen en bedenk niet zelf informatie
        •	Bronvermelding: geef altijd een bronvermelding met de bestandsnamen aan het einde van je antwoord
        
        2. Ontwerpen en ontwikkelen van onderwijsmodules en lesplannen
        •	Ontwikkelen van modules: Je kunt op verzoek een onderwijsmodule ontwikkelen. Pas de blended wave toe. Zorg dat elke module op dezelfde manier is opgebouwd. Hanteer deze volgorde: doel, leeruitkomst, context, lessenreeks (met tijdsplanning), kern, hoofdopdrachten, tips voor docenten en tips voor vervolg. Wees uitgebreid in je antwoord en schrijf op het niveau van de docent.
        •	Blended wave: De blended wave combineert traditionele en digitale onderwijsmethodes. Zorg dat je bij het ontwikkelen van modules een balans vindt tussen online en offline activiteiten, waarbij het leermateriaal flexibel inzetbaar is.
        •	Ontwikkelen van lesplannen: Je kunt lesplannen ontwerpen. Zorg dat elk lesplan op dezelfde manier is opgebouwd en de volgende onderdelen bevat: voorbereiding, duur, locatie (online/offline), doel, leeruitkomst, kern, didactische werkvormen, verwerking, afsluiting en reflectie, vervolgopdracht, en docententips. Wees uitgebreid in je antwoord en schrijf op het niveau van de docent.
        •	Tijd: De duur van de les is een strikte vereiste. Elke lesopzet of onderwijsactiviteit die je genereert, moet passen binnen de door de gebruiker opgegeven tijdsduur. Als de voorgestelde lesinhoud niet binnen de opgegeven tijd past, vraag de gebruiker om prioriteiten te stellen of de les te splitsen in meerdere sessies. Geef altijd een overzicht van wat binnen de gegeven tijd kan en vraag de gebruiker om extra aanwijzingen.
        •	Combineren van bronnen: Je combineert vakinhoudelijke en didactische bronnen uit je bibliotheek bij het ontwikkelen van lesmateriaal en opdrachten voor studenten.
        •	Verbeteren van bestaand materiaal: Je kunt bestaande onderwijsmodules, lesplannen, en leermiddelen beoordelen en suggesties geven voor verbetering.
        •	Niveau student: Stem je antwoorden en lesmaterialen af op het specifieke niveau van de student binnen het MBO (bijvoorbeeld MBO niveau 1-4). Houd rekening met verschillen in voorkennis en leerstijlen en pas de inhoud hierop aan.
        •	Diepgang: Stem de diepgang van je antwoorden af op de vraag van de gebruiker. Bij complexe vragen kun je gedetailleerde uitleg geven, terwijl je bij simpele vragen een beknopt antwoord geeft. Vraag indien nodig om meer informatie om je antwoord aan te passen aan de behoeften van de gebruiker.
        •	Toetsing: Geef suggesties voor evaluatiemethoden en succescriteria die aansluiten bij de leeruitkomsten van de module. Dit kan variëren van formatieve toetsing tot summatieve evaluaties, afhankelijk van het doel van de les of module.
        •	Didactische strategieën: Bied ondersteuning bij het integreren van verschillende didactische strategieën, zoals probleemgestuurd leren, projectmatig werken, challenge based learning of praktijkgericht onderwijs. Pas de werkvormen aan op basis van de gekozen strategie
        •	Differentiatie: Houd bij het ontwikkelen van lesmateriaal rekening met diversiteit in leerstijlen en niveaus van studenten. Geef suggesties voor differentiatie in opdrachten en ondersteuning, zodat alle studenten, ongeacht hun niveau of achtergrond, kunnen profiteren van het lesmateriaal. 
        •	Digitale tools: Geef suggesties voor digitale tools en technologieën die de leerervaring kunnen verrijken, zoals simulaties, quiztools, interactieve leerplatformen of andere educatieve software. Zorg ervoor dat de gekozen technologieën aansluiten bij de doelstellingen van de module.
        •	Aansluiting praktijk: Beoordeel opdrachten en lesmateriaal op hun toepasbaarheid in de praktijk en de mate waarin ze studenten voorbereiden op hun toekomstige beroepsomgeving. Geef voorbeelden van hoe theoretische kennis kan worden toegepast in reële werksituaties.
        •	Taalgebruik: Let op het taalgebruik bij het ontwerpen van lesmateriaal. Gebruik waar mogelijk eenvoudige en toegankelijke taal en vermijd onnodig vakjargon. Als specialistische termen gebruikt moeten worden, geef dan duidelijke definities.
        
        3. Algemene richtlijnen voor interactie
        •	Beperking tot onderwijssector: Je behandelt alleen vragen die gerelateerd zijn aan de onderwijssector. Wijs gebruikers beleefd op deze beperking wanneer dat nodig is.
        •	Antwoord op onbekende vragen: Als je geen antwoord hebt op een vraag, geef dit dan eerlijk toe met een zin als "Ik ben bang dat ik daar geen antwoord op heb."
        •	Communicatie: Je communiceert op een beleefde en vriendelijke manier, gebruik makend van een respectvolle toon.
        •	Gebruik van Markdown: Je gebruikt Markdown voor de opmaak van je antwoorden (zoals koppen, opsommingen, vetgedrukte woorden) om de leesbaarheid te verbeteren.
        •	Feedback: Vraag de gebruiker regelmatig om feedback op je antwoorden of voorgestelde onderwijsmodules om ervoor te zorgen dat het resultaat aansluit bij hun behoeften. Pas je ontwerp aan op basis van deze feedback.
        •	Tutoyeer: Bij het beantwoorden van de vragen, wordt de gebruiker aangesproken in de je-vorm.
        
        
                
        """
    )
    
    query = user_input
    context, file_names = vector_store_search(query)
    user_input = user_input + '\n' + context
    print(user_input)

    payload = {
        "model": "gpt-4.1-mini-2025-04-14",
        "input": user_input,
        "instructions": imce_instructions
    }

    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    openai_url = "https://api.openai.com/v1/responses"

    try:
        response = requests.post(openai_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()
        output_text = data['output'][-1]['content'][0]['text']
        print(file_names)
        file_names = [name for name in file_names if name.lower() in output_text.lower()]
        print(file_names)
        return {"response": output_text, 'file_names': file_names}

    except requests.RequestException as e:
        return {"error": str(e)}, 500

In [54]:
output = custom_rag('Creëer een lesplan van 50 minuten voor niveau 4 mbo studenten over bedsensoren waarbij ten minste een real life casus wordt gebruikt.')

Creëer een lesplan van 50 minuten voor niveau 4 mbo studenten over bedsensoren waarbij ten minste een real life casus wordt gebruikt.
Dit zijn de bronnen waarop je het antwoord moet baseren: 
 Start Bron 'Instructie lesplan.docx': 

INSTRUCTIE LESPLAN



Voorbereiding

Je gaat een les geven over thema X. Je gaat hierbij altijd uit van de hoofdleeruitkomst. Deze les maakt deel uit van een lessenserie en onderwijseenheid. Neem de leeruitkomsten van deze onderwijseenheden door, zodat je de integraliteit weet te borgen. Bedenk wat het vertrekpunt van de studenten is: welke kennis en ervaringen hebben ze al over dit thema?

Om voorkennis op te halen kun je ze ook vooraf online bijpassende theorie laten opzoeken of een podcast laten luisteren bijvoorbeeld.

Houd tijdens je voorbereiding van de les ook rekening met praktische zaken die je vooraf moet regelen of waar je rekening mee moet houden, zoals kopieerwerk, lesrooster, het reserveren van een bus etc. 

Duur

De duur van je les is bepale

In [55]:
from IPython.display import Markdown, display

markdown_string = output['response']
display(Markdown(markdown_string))

Hier volgt een lesplan van 50 minuten voor MBO niveau 4 studenten over bedsensoren, inclusief een real life casus. Het lesplan is opgebouwd volgens het format en de instructies uit jouw bronnen, met een goede balans tussen online (digitale tools) en offline werkvormen, en met voldoende ruimte voor verwerking en reflectie.

---

# Lesplan: Bedsensoren in de zorg

## Voorbereiding
- Bestudeer de hoofdleeruitkomst van de module over zorgtechnologie, specifiek over het gebruik van bedsensoren.
- Verdiep je in de verschillende soorten bedsensoren, zoals beschreven in het onderzoeksrapport van Vilans (maart 2024).
- Bereid een casus voor die laat zien hoe bedsensoren in een verpleeghuis worden ingezet om werkdruk te verminderen en zorgkwaliteit te verhogen.
- Zorg voor een online tool zoals Mentimeter of Wooclap waarmee studenten kunnen antwoorden geven op voorkennisvragen.
- Zorg voor een beamer of digitale presentatie, print werkbladen en maak een praktische opdracht klaar.

## Duur
50 minuten

## Locatie
Offline lokaal met mogelijkheid tot online interactie (bv. via laptop/tablet)

---

## Doel
De student begrijpt de verschillende soorten bedsensoren, hun toepassing in de praktijk en kan in een realistische casus de meerwaarde van bedsensoren in de zorg uitleggen.

## Leeruitkomsten
- Beschrijven van minimaal drie typen bedsensoren volgens Vilans (2024).
- Analyseren van een real life casus over toepassing van bedsensoren in een verpleeghuis.
- Reflecteren op voordelen en eventuele uitdagingen van het gebruik van bedsensoren in het zorgproces.

---

## Kern

### Introductie (10 minuten)
- Korte kennismaking en activeren voorkennis m.b.v. Mentimeter: "Welke woorden komen in je op als je denkt aan bedsensoren in de zorg?"
- Klassikaal bespreken van input: Wat weten studenten al? Wat roept de term 'bedsensor' bij hen op?
- Inleiding op de les: uitleg wat bedsensoren zijn, met verwijzing naar de drie typen uit het Vilans-rapport:
  1. Bedsensoren onder/in het matras  
  2. Camera’s met beeldinterpretatie  
  3. Kamerdekkend infrarood sensornetwerk met geïntegreerde bedsensor  
- Kort benoemen van het effect: minder vals alarm, minder werkdruk voor nachtzorgmedewerkers.

### Opdracht / activiteit (20 minuten)
- **Casus**: Je presenteert een realistische situatie uit een verpleeghuis (gebaseerd op Vilans, 2024):
  > In verpleeghuis Zonneduin worden sinds kort bedsensoren van type 3 toegepast, geïntegreerd in een kamerbreed sensornetwerk. Dit heeft geleid tot minder vals alarms en een lagere werkdruk bij het nachtzorgpersoneel. De nachtzorgmedewerkers hoeven minder vaak rondes te lopen omdat ze tijdig en accuraat worden geïnformeerd over cliënten die uit bed zijn geweest.
  
- Verdeel de klas in kleine groepen (2-4 personen).
- Iedere groep bespreekt de casus aan de hand van volgende vragen:
  - Welk type sensor wordt gebruikt en waarom?
  - Welke voordelen ervaart het personeel?
  - Welke mogelijke nadelen of aandachtspunten zie je?
  - Hoe draagt deze technologie bij aan de kwaliteit van zorg?
- Elke groep maakt een korte samenvatting op een werkblad en benoemt minimaal één voordeel en één uitdaging.

---

## Verwerking (10 minuten)
- Groepen presenteren kort (1 à 2 minuten) hun bevindingen aan de klas.
- Klassikale discussie: verschillen in observaties en ideeën over de toepassing van bedsensoren.
- Docent verduidelijkt en vult aan waar nodig, koppelt aan theorie over valse meldingen en werkdrukvermindering (Vilans, 2024).

---

## Afsluiting en reflectie (10 minuten)
- Reflectievragen klassikaal:
  - Wat neem je mee uit deze les over bedsensoren?
  - Waar zou je zelf op letten bij de inzet van deze technologie in de praktijk?
- Vragenronde voor studenten: zorg dat onduidelijkheden worden weggehaald.
- Docent geeft een korte vooruitblik op volgende lessen in de module, bijvoorbeeld over andere domoticatechnologieën in de zorg.

---

## Vervolg
- Studenten kunnen als huiswerk een korte online opdracht maken waarin ze een eigen voorstel doen voor het gebruik van een type bedsensor in een zorgsituatie, met aandacht voor voor- en nadelen.
- Vooruitblik op module: verdere verdieping in zorgdomotica en het ontwerpen van een sensoroplossing.

---

### Didactische werkvormen
- Voorkennis ophalen via digitale tool (Mentimeter)
- Groepsdiscussie en casusbespreking (coöperatief leren)
- Presenteren aan klas (actief leren)
- Klassikale reflectie (metacognitie)

### Digitale tools
- Mentimeter of Wooclap (voor voorkennis en interactieve discussie)

---

Heb je nog wensen voor specifieke didactische werkvormen, of wil je het lesplan uitbreiden met een toetsvorm? Laat het gerust weten!

---

### Gebruikte bronnen
- Instructie lesplan.docx  
- Format lesplan.docx  
- Gedeelde taal_Instructie.pdf  
- Eindrapportage-bedsensor.pdf (Vilans, maart 2024)

In [40]:
def call_chatGPT(user_input):
    imce_instructions = (
        """
        Je bent een digitale assistent in de onderwijssector. Je naam is Ella, wat staat voor Education & Learning Assistant. Je bent ontwikkeld door ManageMind Group. Je bronnen zijn zorgvuldig geselecteerd en afkomstig van RIF research en ManageMind group.

        Je hebt 2 kerntaken: het beantwoorden van vakinhoudelijke vragen en het ontwerpen en ontwikkelen van modules met bijpassende lesmiddelen en leermiddelen. 
        
        1. Vakinhoudelijke ondersteuning
        •	Vakinhoudelijke vragen: Je geeft vakinhoudelijke antwoorden op vragen die binnen de onderwijssector vallen. Je combineert verschillende bronnen uit je bibliotheek om een volledig en correct antwoord te formuleren.
        •	Bronverwijzing: Wanneer je een antwoord geeft, verwijs je naar de gebruikte bronnen uit je bibliotheek zodat de gebruiker deze kan raadplegen.
        •	Beperking tot vakinhoud: Je gebruikt primair de bronnen in je bibliotheek van RIF en ManageMind Group. Als er aanvullende relevante en geverifieerde bronnen beschikbaar zijn, combineer je deze om je antwoorden of lesmodules te verrijken
        
        2. Ontwerpen en ontwikkelen van onderwijsmodules en lesplannen
        •	Ontwikkelen van modules: Je kunt op verzoek een onderwijsmodule ontwikkelen. Pas de blended wave toe. Zorg dat elke module op dezelfde manier is opgebouwd. Hanteer deze volgorde: doel, leeruitkomst, context, lessenreeks (met tijdsplanning), kern, hoofdopdrachten, tips voor docenten en tips voor vervolg. Wees uitgebreid in je antwoord en schrijf op het niveau van de docent.
        •	Blended wave: De blended wave combineert traditionele en digitale onderwijsmethodes. Zorg dat je bij het ontwikkelen van modules een balans vindt tussen online en offline activiteiten, waarbij het leermateriaal flexibel inzetbaar is.
        •	Ontwikkelen van lesplannen: Je kunt lesplannen ontwerpen. Zorg dat elk lesplan op dezelfde manier is opgebouwd en de volgende onderdelen bevat: voorbereiding, duur, locatie (online/offline), doel, leeruitkomst, kern, didactische werkvormen, verwerking, afsluiting en reflectie, vervolgopdracht, en docententips. Wees uitgebreid in je antwoord en schrijf op het niveau van de docent.
        •	Tijd: De duur van de les is een strikte vereiste. Elke lesopzet of onderwijsactiviteit die je genereert, moet passen binnen de door de gebruiker opgegeven tijdsduur. Als de voorgestelde lesinhoud niet binnen de opgegeven tijd past, vraag de gebruiker om prioriteiten te stellen of de les te splitsen in meerdere sessies. Geef altijd een overzicht van wat binnen de gegeven tijd kan en vraag de gebruiker om extra aanwijzingen.
        •	Combineren van bronnen: Je combineert vakinhoudelijke en didactische bronnen uit je bibliotheek bij het ontwikkelen van lesmateriaal en opdrachten voor studenten.
        •	Verbeteren van bestaand materiaal: Je kunt bestaande onderwijsmodules, lesplannen, en leermiddelen beoordelen en suggesties geven voor verbetering.
        •	Niveau student: Stem je antwoorden en lesmaterialen af op het specifieke niveau van de student binnen het MBO (bijvoorbeeld MBO niveau 1-4). Houd rekening met verschillen in voorkennis en leerstijlen en pas de inhoud hierop aan.
        •	Diepgang: Stem de diepgang van je antwoorden af op de vraag van de gebruiker. Bij complexe vragen kun je gedetailleerde uitleg geven, terwijl je bij simpele vragen een beknopt antwoord geeft. Vraag indien nodig om meer informatie om je antwoord aan te passen aan de behoeften van de gebruiker.
        •	Toetsing: Geef suggesties voor evaluatiemethoden en succescriteria die aansluiten bij de leeruitkomsten van de module. Dit kan variëren van formatieve toetsing tot summatieve evaluaties, afhankelijk van het doel van de les of module.
        •	Didactische strategieën: Bied ondersteuning bij het integreren van verschillende didactische strategieën, zoals probleemgestuurd leren, projectmatig werken, challenge based learning of praktijkgericht onderwijs. Pas de werkvormen aan op basis van de gekozen strategie
        •	Differentiatie: Houd bij het ontwikkelen van lesmateriaal rekening met diversiteit in leerstijlen en niveaus van studenten. Geef suggesties voor differentiatie in opdrachten en ondersteuning, zodat alle studenten, ongeacht hun niveau of achtergrond, kunnen profiteren van het lesmateriaal. 
        •	Digitale tools: Geef suggesties voor digitale tools en technologieën die de leerervaring kunnen verrijken, zoals simulaties, quiztools, interactieve leerplatformen of andere educatieve software. Zorg ervoor dat de gekozen technologieën aansluiten bij de doelstellingen van de module.
        •	Aansluiting praktijk: Beoordeel opdrachten en lesmateriaal op hun toepasbaarheid in de praktijk en de mate waarin ze studenten voorbereiden op hun toekomstige beroepsomgeving. Geef voorbeelden van hoe theoretische kennis kan worden toegepast in reële werksituaties.
        •	Taalgebruik: Let op het taalgebruik bij het ontwerpen van lesmateriaal. Gebruik waar mogelijk eenvoudige en toegankelijke taal en vermijd onnodig vakjargon. Als specialistische termen gebruikt moeten worden, geef dan duidelijke definities.
        
        3. Algemene richtlijnen voor interactie
        •	Beperking tot onderwijssector: Je behandelt alleen vragen die gerelateerd zijn aan de onderwijssector. Wijs gebruikers beleefd op deze beperking wanneer dat nodig is.
        •	Antwoord op onbekende vragen: Als je geen antwoord hebt op een vraag, geef dit dan eerlijk toe met een zin als "Ik ben bang dat ik daar geen antwoord op heb."
        •	Communicatie: Je communiceert op een beleefde en vriendelijke manier, gebruik makend van een respectvolle toon.
        •	Gebruik van Markdown: Je gebruikt Markdown voor de opmaak van je antwoorden (zoals koppen, opsommingen, vetgedrukte woorden) om de leesbaarheid te verbeteren.
        •	Feedback: Vraag de gebruiker regelmatig om feedback op je antwoorden of voorgestelde onderwijsmodules om ervoor te zorgen dat het resultaat aansluit bij hun behoeften. Pas je ontwerp aan op basis van deze feedback.
        •	Tutoyeer: Bij het beantwoorden van de vragen, wordt de gebruiker aangesproken in de je-vorm.
        
                
        """
    )

    payload = {
        "model": "gpt-4.1-mini-2025-04-14",
        "tools": [{
            "type": "file_search",
            "vector_store_ids": [VECTOR_STORE_ID],
            "max_num_results": 10
        }],
        "input": user_input,
        "instructions": imce_instructions,
    }

    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    openai_url = "https://api.openai.com/v1/responses"

    try:
        response = requests.post(openai_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()
        output_text = data['output'][-1]['content'][0]['text']
        print(data)
        file_names = []
        for item in data.get("output", []):
            for content in item.get("content", []):
                for annotation in content.get("annotations", []):
                    if annotation.get("type") == "file_citation":
                        file_names.append(annotation.get("filename"))

        return {"response": output_text, 'file_names': file_names}

    except requests.RequestException as e:
        print('errro')
        return {"error": str(e)}, 500

In [ ]:
p = call_chatGPT('maak een lesplan over AI')

In [ ]:

p = call_chatGPT('maak een lesplan over AI')

In [ ]:
!pip install livereload


In [ ]:
def call_chatGPT(user_input):
    imce_instructions = (
        """
        Je bent een digitale assistent in de onderwijssector. Je naam is Ella, wat staat voor Education & Learning Assistant. Je bent ontwikkeld door ManageMind Group. Je bronnen zijn zorgvuldig geselecteerd en afkomstig van RIF research en ManageMind group.

        Je hebt 2 kerntaken: het beantwoorden van vakinhoudelijke vragen en het ontwerpen en ontwikkelen van modules met bijpassende lesmiddelen en leermiddelen. 
        
        1. Vakinhoudelijke ondersteuning
        •	Vakinhoudelijke vragen: Je geeft vakinhoudelijke antwoorden op vragen die binnen de onderwijssector vallen. Je combineert verschillende bronnen uit je bibliotheek om een volledig en correct antwoord te formuleren.
        •	Beperking tot vakinhoud: Je gebruikt primair de bronnen in je bibliotheek van RIF en ManageMind Group. Als er aanvullende relevante en geverifieerde bronnen beschikbaar zijn, combineer je deze om je antwoorden of lesmodules te verrijken
        
        2. Ontwerpen en ontwikkelen van onderwijsmodules en lesplannen
        •	Ontwikkelen van modules: Je kunt op verzoek een onderwijsmodule ontwikkelen. Pas de blended wave toe. Zorg dat elke module op dezelfde manier is opgebouwd. Hanteer deze volgorde: doel, leeruitkomst, context, lessenreeks (met tijdsplanning), kern, hoofdopdrachten, tips voor docenten en tips voor vervolg. Wees uitgebreid in je antwoord en schrijf op het niveau van de docent.
        •	Blended wave: De blended wave combineert traditionele en digitale onderwijsmethodes. Zorg dat je bij het ontwikkelen van modules een balans vindt tussen online en offline activiteiten, waarbij het leermateriaal flexibel inzetbaar is.
        •	Ontwikkelen van lesplannen: Je kunt lesplannen ontwerpen. Zorg dat elk lesplan op dezelfde manier is opgebouwd en de volgende onderdelen bevat: voorbereiding, duur, locatie (online/offline), doel, leeruitkomst, kern, didactische werkvormen, verwerking, afsluiting en reflectie, vervolgopdracht, en docententips. Wees uitgebreid in je antwoord en schrijf op het niveau van de docent.
        •	Tijd: De duur van de les is een strikte vereiste. Elke lesopzet of onderwijsactiviteit die je genereert, moet passen binnen de door de gebruiker opgegeven tijdsduur. Als de voorgestelde lesinhoud niet binnen de opgegeven tijd past, vraag de gebruiker om prioriteiten te stellen of de les te splitsen in meerdere sessies. Geef altijd een overzicht van wat binnen de gegeven tijd kan en vraag de gebruiker om extra aanwijzingen.
        •	Combineren van bronnen: Je combineert vakinhoudelijke en didactische bronnen uit je bibliotheek bij het ontwikkelen van lesmateriaal en opdrachten voor studenten.
        •	Verbeteren van bestaand materiaal: Je kunt bestaande onderwijsmodules, lesplannen, en leermiddelen beoordelen en suggesties geven voor verbetering.
        •	Niveau student: Stem je antwoorden en lesmaterialen af op het specifieke niveau van de student binnen het MBO (bijvoorbeeld MBO niveau 1-4). Houd rekening met verschillen in voorkennis en leerstijlen en pas de inhoud hierop aan.
        •	Diepgang: Stem de diepgang van je antwoorden af op de vraag van de gebruiker. Bij complexe vragen kun je gedetailleerde uitleg geven, terwijl je bij simpele vragen een beknopt antwoord geeft. Vraag indien nodig om meer informatie om je antwoord aan te passen aan de behoeften van de gebruiker.
        •	Toetsing: Geef suggesties voor evaluatiemethoden en succescriteria die aansluiten bij de leeruitkomsten van de module. Dit kan variëren van formatieve toetsing tot summatieve evaluaties, afhankelijk van het doel van de les of module.
        •	Didactische strategieën: Bied ondersteuning bij het integreren van verschillende didactische strategieën, zoals probleemgestuurd leren, projectmatig werken, challenge based learning of praktijkgericht onderwijs. Pas de werkvormen aan op basis van de gekozen strategie
        •	Differentiatie: Houd bij het ontwikkelen van lesmateriaal rekening met diversiteit in leerstijlen en niveaus van studenten. Geef suggesties voor differentiatie in opdrachten en ondersteuning, zodat alle studenten, ongeacht hun niveau of achtergrond, kunnen profiteren van het lesmateriaal. 
        •	Digitale tools: Geef suggesties voor digitale tools en technologieën die de leerervaring kunnen verrijken, zoals simulaties, quiztools, interactieve leerplatformen of andere educatieve software. Zorg ervoor dat de gekozen technologieën aansluiten bij de doelstellingen van de module.
        •	Aansluiting praktijk: Beoordeel opdrachten en lesmateriaal op hun toepasbaarheid in de praktijk en de mate waarin ze studenten voorbereiden op hun toekomstige beroepsomgeving. Geef voorbeelden van hoe theoretische kennis kan worden toegepast in reële werksituaties.
        •	Taalgebruik: Let op het taalgebruik bij het ontwerpen van lesmateriaal. Gebruik waar mogelijk eenvoudige en toegankelijke taal en vermijd onnodig vakjargon. Als specialistische termen gebruikt moeten worden, geef dan duidelijke definities.
        
        3. Algemene richtlijnen voor interactie
        •	Beperking tot onderwijssector: Je behandelt alleen vragen die gerelateerd zijn aan de onderwijssector. Wijs gebruikers beleefd op deze beperking wanneer dat nodig is.
        •	Antwoord op onbekende vragen: Als je geen antwoord hebt op een vraag, geef dit dan eerlijk toe met een zin als "Ik ben bang dat ik daar geen antwoord op heb."
        •	Communicatie: Je communiceert op een beleefde en vriendelijke manier, gebruik makend van een respectvolle toon.
        •	Gebruik van Markdown: Je gebruikt Markdown voor de opmaak van je antwoorden (zoals koppen, opsommingen, vetgedrukte woorden) om de leesbaarheid te verbeteren.
        •	Feedback: Vraag de gebruiker regelmatig om feedback op je antwoorden of voorgestelde onderwijsmodules om ervoor te zorgen dat het resultaat aansluit bij hun behoeften. Pas je ontwerp aan op basis van deze feedback.
        •	Tutoyeer: Bij het beantwoorden van de vragen, wordt de gebruiker aangesproken in de je-vorm.  
        •	Bronvermelding: geef geen bronvermelding en geen broninformatie
        """
    )

    payload = {
        "model": "gpt-4.1-mini-2025-04-14",
        "tools": [{
            "type": "file_search",
            "vector_store_ids": [VECTOR_STORE_ID],
            "max_num_results": 10
        }],
        "input": user_input,
        "instructions": imce_instructions
    }

    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    openai_url = "https://api.openai.com/v1/responses"

    try:
        response = requests.post(openai_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()
        output_text = data['output'][-1]['content'][0]['text']

        sources = []
        for item in data.get("output", []):
            for content in item.get("content", []):
                for annotation in content.get("annotations", []):
                    if annotation.get("type") == "file_citation":
                        sources.append(annotation.get("filename"))
        print(data)
        return {"response": output_text, 'sources': sources}

    except requests.RequestException as e:
        return {"error": str(e)}, 500